

```
# Google Dialogflow - Intents
```



In [ ]:
# receives
# .txt file containing a list of question-answer pairs identified by 'P: ' and 'R: '
# .txt file with a list of questions, one question per line

# retrieves
# .txt file containing the posed questions and respective Dialogflow's answers, identified by 'P: ' and 'R: '

In [ ]:
pip install google-cloud-dialogflow

In [ ]:
pip install google-api-core==1.28.0

In [7]:
from google.cloud import dialogflow_v2beta1 as dialogflow
from google.api_core.exceptions import InvalidArgument
import time

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# google cloud project credentials
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = # path to json file with credentials
!echo $GOOGLE_APPLICATION_CREDENTIALS

In [4]:
# file management
def open_file(filename):
    read_file = open(filename, 'r')
    file_cont = read_file.readlines()
    read_file.close()

    return file_cont

def write_file(filename, content):
    file_write = open(filename, 'w')
    file_write.writelines(content)
    file_write.close()

In [5]:
# creates an intent with the given training phrase and answer
def create_intent(project_id, intent_name, training_phrase, answer):
    language = 'pt'

    intents_client = dialogflow.IntentsClient()
    parent = dialogflow.AgentsClient.agent_path(project_id)
    training_phrases = []

    # create a training phrase
    part = dialogflow.Intent.TrainingPhrase.Part(text=training_phrase)
    training_phrase = dialogflow.Intent.TrainingPhrase(parts=[part])
    training_phrases.append(training_phrase)

    # create an answer
    text = dialogflow.Intent.Message.Text(text=[answer])
    message = dialogflow.Intent.Message(text=text)

    # create the intent
    intent = dialogflow.Intent(display_name=intent_name, training_phrases=training_phrases, messages=[message])
    response = intents_client.create_intent(request={"parent": parent, "intent": intent})

In [6]:
# creates an intent per training phrase in list
def create_all_intents(project_id, intents_list, training_phrases_list, answers_list):

    # due to usage limitation, wait 30 seconds for each 10 created intents
    for i in range(len(intents_list)):
        if i >= 1:
            if i % 10 == 0:
                print('... waiting ...')
                time.sleep(30)
        
        # create the intent   
        create_intent(project_id, intents_list[i], training_phrases_list[i], answers_list[i])
        print('Created Intent: ', intents_list[i])

    print('All intents created!')

In [10]:
# returns the answer corresponding to the question in position question_index of question-answer pairs file
def get_answer(file_content, question_index):
    answer = ''

    for i in range(question_index + 1, len(file_content)):
        if file_content[i] == '\n' or 'P: ' in file_content[i]:
            break
        else:
            answer = answer + file_content[i]

    return answer

In [15]:
# divides questions and answers in lists of training phrases and answers, to create intents
# the same position in each list represents the question-answer pair
def prepare_file(project_id, domain_file_path):
    faqs_list = open_file(domain_file_path)
    intents_list = []
    training_phrases_list = []
    answers_list = []
    count = 0

    # for each 'P: ' found, adds the respective question and answer to training phrase and answer lists
    for i in range(len(faqs_list)):
        if 'P: ' in faqs_list[i]:
            count+=1

            # name of each intent
            intents_list.append('Intent' + str(count))
            aux_question = faqs_list[i].replace('P: ', '')
            
            # maximum lenght for a training phrase
            if len(aux_question) > 768:
                aux_question = aux_question[:767]

            # adds question to training phrase list
            training_phrases_list.append(aux_question)

            # adds respective answer to answers list
            answer = get_answer(faqs_list, i)
            aux_answer = answer.replace('R: ', '')
            answers_list.append(aux_answer)
                
        else:
            continue

    return intents_list, training_phrases_list, answers_list
    # create_all_intents(project_id, intents_list, training_phrases_list, responses_list)

In [12]:
# given a question, retrieves Dialogflow's answer
def retrieve_answer(project_id, language, session_id, question):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(project_id, session_id)

    text_input = dialogflow.TextInput(text=question, language_code=language)
    query_input = dialogflow.QueryInput(text=text_input)

    try:
        answer = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
        raise

    return answer

In [20]:
# retrieves a file with the posed questions and Dialogflows's answers
def testing(project_id, language, session_id, input_questions_file_path, save_file_path):
    input_questions = open_file(input_questions_file_path)
    final_file_content = []

    for i in range(len(input_questions)):
        # due to usage limitation, wait 30 seconds for each 10 retrieved answers
        if i >= 1:
            if i % 10 == 0:
                print('... waiting ...')
                time.sleep(30)

        # maximum lenght for an input question
        aux_question = input_questions[i]
        if len(aux_question) > 256:
            aux_question = aux_question[0:256]

        # retrieves answer to input question
        answer = retrieve_answer(project_id, language, session_id, aux_question)
        print('Response %d retrieved' % (i+1))

        # creates the final file containing all posed questions and respective retrieved answers, with 'P: ' and 'R: ' identifiers
        aux_q = 'P: ' + aux_question
        final_file_content.append(aux_q)

        aux_a = 'R: ' + answer.query_result.fulfillment_text
        final_file_content.append(aux_a)
        final_file_content.append('\n')

    write_file(save_file_path, final_file_content)
    print('File with posed questions and respective answers created!')

In [ ]:
# NOTES

# prior to running the code, google cloud credentials and a project must be created on the google cloud platform
# https://console.cloud.google.com/?hl=pt

# in addition, a google dialogflow agent must be created on the dialogflow platform
# https://dialogflow.cloud.google.com/

# domain_file_path - path to the file containing the domain
# must be a file containing question-answer pairs identified with 'P: ' and 'R: ', respectively
# FAQs                      
# P: question1             
# R: answer1           
# \n                        
# P: question2              
# R: answer2                
# \n                       
# must be a .txt file

# questions_file_path - path to the file containing all questions, one question per line
# Q1
# Q2
# Q3
# ...
# must be a .txt file

# save_file_path - path to the file where the posed questions and respective retrieved answers are to be saved
# must be a .txt file

In [ ]:
project_id = # 'project-id' retrieved from google cloud project, created in google cloud platform
session_id = '1234567890'
language = 'pt'

domain_file_path = # 'domain_file_path'
input_questions_file_path = # 'input_questions_file_path'
save_file_path = # 'save_file_path'

(intents_list, training_phrases_list, answers_list) = prepare_file(project_id, domain_file_path)
create_all_intents(project_id, intents_list, training_phrases_list, answers_list)
testing(project_id, language, session_id, input_questions_file_path, save_file_path)